## Imports

In [1]:
from article_recommender import calculate_candidate_embeddings, load_model

/opt/homebrew/Caskroom/miniconda/base/envs/hermes/lib/python3.13/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/hermes/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# connect to the database
import pymongo
from pymongo import MongoClient
client = MongoClient("mongodb://localhost:27017/")
db = client["hermes"]
collection = db["articles"]

# loop though the articles and update the topic
total = collection.count_documents({"embeddings": {"$exists": False}})
print(f"Total articles: {total}")
model = load_model()

Total articles: 125107


In [8]:
from tqdm import tqdm

# loop through the articles and update the topic in batches of 64 articles
batch_size = 512

for i in tqdm(range(0, total, batch_size), desc="Processing batches"):
    articles_cursor = collection.find(
        {"embeddings": {"$exists": False}}).limit(batch_size)
    articles = list(articles_cursor)
    titles = [article["title"] for article in articles]

    results = calculate_candidate_embeddings(model, titles)
    for article, result in zip(articles, results):

        collection.update_one({"_id": article["_id"]}, {
                              "$set": {"embeddings": result.detach().cpu().numpy().tolist()}})

Processing batches: 100%|██████████| 245/245 [04:16<00:00,  1.05s/it]
